#### Coutresy to convert XML to JSON https://stackoverflow.com/questions/191536/converting-xml-to-json-using-python 
Made small modifications for our purpose
Remove certain first few lines from XML files and run the code by changing the XML filename

In [24]:
from xml.dom import minidom
import simplejson as json
def parse_element(element):
    dict_data = dict()
    if element.nodeType == element.TEXT_NODE:
        if element.data != " ":
            dict_data['data'] = element.data
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_NODE, 
                                element.DOCUMENT_TYPE_NODE]:
        for item in element.attributes.items():
            dict_data[item[0]] = item[1]
    if element.nodeType not in [element.TEXT_NODE, element.DOCUMENT_TYPE_NODE]:
        for child in element.childNodes:
            child_name, child_dict = parse_element(child)
            if child_name in dict_data:
                try:
                    dict_data[child_name].append(child_dict)
                except AttributeError:
                    dict_data[child_name] = [dict_data[child_name], child_dict]
            else:
                dict_data[child_name] = child_dict 
    return element.nodeName, dict_data

if __name__ == '__main__':
    dom = minidom.parse('Q2Q.xml')
    print type(dom)
    f = open('Q2Q.json', 'w')
    f.write(json.dumps(parse_element(dom), sort_keys=True, indent=4))
    f.close()

<type 'instance'>


In [25]:
import pandas as pd

data111 = pd.read_json('Q2Q.json')
data111.head()
print data111[0][1]['xml'].keys()
df3111 = pd.io.json.json_normalize(data111[0][1]['xml']['OrgQuestion'])
df3111.head()
df4111 = pd.concat([pd.DataFrame(pd.io.json.json_normalize(x)) for x in df3111['Thread.RelComment']],ignore_index=True)
df4111['THREAD_SEQUENCE'] = [ '_'.join(i.split('_')[:2]) for i in df4111['RELC_ID']]
print df4111.count()
df4111.head()

[u'OrgQuestion', u'#text', u'version']
#text                   5000
RELC_DATE               5000
RELC_ID                 5000
RELC_RELEVANCE2ORGQ     5000
RELC_RELEVANCE2RELQ     5000
RELC_USERID             5000
RELC_USERNAME           5000
RelCBody.#text.data     5000
RelCClean.#text.data    5000
THREAD_SEQUENCE         5000
dtype: int64


,#text,RELC_DATE,RELC_ID,RELC_RELEVANCE2ORGQ,RELC_RELEVANCE2RELQ,RELC_USERID,RELC_USERNAME,RelCBody.#text.data,RelCClean.#text.data,THREAD_SEQUENCE
0,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 07:23:20,Q268_R4_C1,Good,Good,U594,Dilgeer,Commercial bank/IBQ,Commercial bank/IBQ,Q268_R4
1,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 12:58:13,Q268_R4_C2,Good,Good,U979,Speedysid,The best bank in Qatar for you would be the on...,The best bank in Qatar for you would be the on...,Q268_R4
2,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 12:58:25,Q268_R4_C3,PotentiallyUseful,PotentiallyUseful,U979,Speedysid,- Credit / Debit card facilities- Customer Car...,- Credit / Debit card facilities - Customer Ca...,Q268_R4
3,"[{u'data': u' '}, {u'data': u' ...",2013-05-03 22:25:47,Q268_R4_C4,Good,Good,U4883,puru1600,Commercial Bank,Commercial Bank,Q268_R4
4,"[{u'data': u' '}, {u'data': u' ...",2013-05-04 07:04:50,Q268_R4_C5,PotentiallyUseful,Good,U4884,usmi,Any bank other than Doha Bank ;),Any bank other than Doha Bank ;),Q268_R4


In [27]:
print df3111.count()
#df3111.describe()

#text                                                  500
ORGQ_ID                                                500
OrgQBody.#text.data                                    500
OrgQClean.#text.data                                   500
OrgQSubject.#text.data                                 500
Thread.#text                                           500
Thread.RelComment                                      500
Thread.RelQuestion.#text                               500
Thread.RelQuestion.RELQ_CATEGORY                       500
Thread.RelQuestion.RELQ_DATE                           500
Thread.RelQuestion.RELQ_ID                             500
Thread.RelQuestion.RELQ_RANKING_ORDER                  500
Thread.RelQuestion.RELQ_RELEVANCE2ORGQ                 500
Thread.RelQuestion.RELQ_USERID                         500
Thread.RelQuestion.RELQ_USERNAME                       500
Thread.RelQuestion.RelQBody.#text.data                 500
Thread.RelQuestion.RelQClean.#text.data                5

In [29]:
df5111 = pd.concat([df3111.set_index('Thread.RelQuestion.RELQ_ID'),df4111.set_index('THREAD_SEQUENCE')], axis=1, join='inner').reset_index()
df5111.columns

remove_index = [ '#text', 'OrgQBody.#text.data' ,'Thread.#text', 'Thread.RelQuestion.#text', 'Thread.RelQuestion.RelQBody.#text.data' 
               ,'Thread.SubtaskA_Skip_Because_Same_As_RelQuestion_ID' , 'index' , 'RelCBody.#text.data' , 'Thread.RelComment']

[ df5111.drop(i, axis=1, inplace=True) for i in remove_index ] 
df5111[ df5111['ORGQ_ID'] == 'Q268' ].columns

Index([u'ORGQ_ID', u'OrgQClean.#text.data', u'OrgQSubject.#text.data',
       u'Thread.RelQuestion.RELQ_CATEGORY', u'Thread.RelQuestion.RELQ_DATE',
       u'Thread.RelQuestion.RELQ_RANKING_ORDER',
       u'Thread.RelQuestion.RELQ_RELEVANCE2ORGQ',
       u'Thread.RelQuestion.RELQ_USERID', u'Thread.RelQuestion.RELQ_USERNAME',
       u'Thread.RelQuestion.RelQClean.#text.data',
       u'Thread.RelQuestion.RelQSubject.#text.data', u'Thread.THREAD_SEQUENCE',
       u'RELC_DATE', u'RELC_ID', u'RELC_RELEVANCE2ORGQ',
       u'RELC_RELEVANCE2RELQ', u'RELC_USERID', u'RELC_USERNAME',
       u'RelCClean.#text.data'],
      dtype='object')

In [38]:
df5111['Thread.RelQuestion.RelQClean.#text.data']

0       Best Bank // Hi Guys; I need to open a new ban...
1       Best Bank // Hi Guys; I need to open a new ban...
2       Best Bank // Hi Guys; I need to open a new ban...
3       Best Bank // Hi Guys; I need to open a new ban...
4       Best Bank // Hi Guys; I need to open a new ban...
5       Best Bank // Hi Guys; I need to open a new ban...
6       Best Bank // Hi Guys; I need to open a new ban...
7       Best Bank // Hi Guys; I need to open a new ban...
8       Best Bank // Hi Guys; I need to open a new ban...
9       Best Bank // Hi Guys; I need to open a new ban...
10      What is the best bank to open an account? // S...
11      What is the best bank to open an account? // S...
12      What is the best bank to open an account? // S...
13      What is the best bank to open an account? // S...
14      What is the best bank to open an account? // S...
15      What is the best bank to open an account? // S...
16      What is the best bank to open an account? // S...
17      What i